# Exploring Urban Heat with ECOSTRESS



## Set up your analysis

In [1]:
# Import Packages
import os
import pathlib
import shutil
import warnings

import cartopy.crs as ccrs
import rioxarray as rxr
import hvplot.xarray
import hvplot.pandas
import holoviews as hv
import geoviews as gv
import geopandas as gpd
from holoviews.selection import link_selections

# Some cells may generate warnings that we can ignore. Comment below lines to see.
warnings.filterwarnings('ignore')

# Set some default plotting options.
size_opts = dict(frame_width=400, fontscale=1.5)
map_opts = dict(
    geo=True, tiles='EsriImagery',
    rot=90,
    xlabel='Longitude', ylabel='Latitude')

In [2]:
"""
Copy the data-store to 'local' directory
This enables quick access to data files
"""

# Identify the location of the HYR-SENSE "data store"
data_store_path = os.path.join(
    f'{os.sep}data-store', 'iplant', 'home', 'shared', 
    'esiil', 'HYR_SENSE', 'data', '01-Urban-Heat-Island')
# Set a destination path (this is a 'local' and temporary path)
# in the GitHub repo we cloned
data_dir = os.path.join(
    pathlib.Path.home(), 'HYR-SENSE', 'data', 'Urban-Heat-Island/')
# create the directory for the copied data, if needed
if not os.path.exists(data_dir):
    os.mkdir(data_dir) 
    
# Using 'shutil' package, copy all the files over
shutil.copytree(data_store_path, data_dir, dirs_exist_ok=True)

'/home/jovyan/HYR-SENSE/data/Urban-Heat-Island/'

## Define the Region of Interest -- City of Boulder

In [3]:
# Import the City of Boulder boundary
# ---

# Define a path to the boundary
boulder_path = os.path.join(
    data_dir,
    # City of Boulder file
    'City_of_Boulder_City_Limits.zip'
)

# Read file and merge geometries
boulder_gdf = gpd.read_file(boulder_path).dissolve()

# Check that we have a GeoDataFrame
boulder_gdf

,geometry,OBJECTID,TYPE,ShapeSTAre,ShapeSTLen
0,"MULTIPOLYGON Z (((-105.21244 40.01732 0.00000,...",38,City,7.138182e+07,45971.753983


### Create a site map

In [4]:
# Plot the boundary
(
    boulder_gdf.hvplot(
        **map_opts,
        line_color='orange', line_width=3,
        fill_color='white', fill_alpha=.05,
    )
    *
    gpd.GeoDataFrame(geometry=boulder_gdf.envelope).hvplot(
        **map_opts,
        line_color='skyblue', fill_color=None
    )
).opts(**size_opts)

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]
   .WMTS.II     :WMTS   [Longitude,Latitude]
   .Polygons.II :Polygons   [Longitude,Latitude]

## Prepare the Land Surface Temperature data

In [5]:
# Import the ECOSTRESS scene
# ---

# Define a filepath for  ECOSTRESS L2T LSTE
eco_path = os.path.join(
    data_dir,
    # ECOSTRESS scene
    'ECOv002_L2T_LSTE_28527_009_13TDE_20230718T081442_0710_01_LST.tif'
)

# Open the LSTE file using open_rasterio from the rioxarray library
eco_lst_da = (
    rxr.open_rasterio(eco_path)
    # There is only 1 band, so we can squeeze and remove the band dimension.
    .squeeze('band', drop=True)
)
eco_lst_da

<xarray.DataArray (y: 1568, x: 1568)>
[2458624 values with dtype=float32]
Coordinates:
  * x            (x) float64 4e+05 4.001e+05 4.001e+05 ... 5.096e+05 5.097e+05
  * y            (y) float64 4.5e+06 4.5e+06 4.5e+06 ... 4.39e+06 4.39e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     nan
    scale_factor:   1.0
    add_offset:     0.0

In [6]:
# Crop ECOSTRESS data to Region of Interest
# ---

boulder_lst_da = (
    eco_lst_da
    .rio.reproject('EPSG:4326')
    .rio.clip(boulder_gdf.to_crs('EPSG:4326').geometry)
)

## Prepare the Tree Data

In [11]:
# Open up the Boulder Tree Inventory data
# ---

# Define a path to the boundary
trees_path = os.path.join(
    data_dir,
    # Tree Inventory File
    'Tree_Inventory_Open_Data.zip'
)

# Read file and merge geometries
trees_gdf = gpd.read_file(trees_path)

trees_gdf = trees_gdf[trees_gdf.geometry!=None]

# Clip to the ROI
trees_gdf = trees_gdf.sjoin(boulder_gdf)

# Check that we have a GeoDataFrame
trees_gdf

,OBJECTID_left,FACILITYID,SPECIESCD,FULLNAME,LATINNAME,GENUS,CULTIVAR,COMMONNAME,GENUSCOM,LEAFCYCLE,...,DATACONF,DIVERSITYI,SPECIESIND,OTHERINDEX,geometry,index_right,OBJECTID_right,TYPE,ShapeSTAre,ShapeSTLen
0,1,TREE79142,FRPE,Fraxinus pennsylvanica,Fraxinus pennsylvanica,Fraxinus,None,"Ash, Green",Ash,Deciduous,...,None,None,None,None,POINT (-105.22100 40.02395),0,38,City,7.138182e+07,45971.753983
1,2,TREE79143,FRPE,Fraxinus pennsylvanica,Fraxinus pennsylvanica,Fraxinus,None,"Ash, Green",Ash,Deciduous,...,None,None,None,None,POINT (-105.22103 40.02378),0,38,City,7.138182e+07,45971.753983
2,3,TREE79144,CASP,Catalpa speciosa,Catalpa speciosa,Catalpa,None,"Catalpa, Western",Catalpa,Deciduous,...,None,None,None,None,POINT (-105.22109 40.02361),0,38,City,7.138182e+07,45971.753983
3,4,TREE79145,CASP,Catalpa speciosa,Catalpa speciosa,Catalpa,None,"Catalpa, Western",Catalpa,Deciduous,...,None,None,None,None,POINT (-105.22116 40.02349),0,38,City,7.138182e+07,45971.753983
4,5,TREE79146,FRAM,Fraxinus americana,Fraxinus americana,Fraxinus,None,"Ash, White",Ash,Deciduous,...,None,None,None,None,POINT (-105.22174 40.02329),0,38,City,7.138182e+07,45971.753983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49823,119846,TREE292192,Unknown,Unknown,Unknown,Unknown,None,"Willow, Crack",Unknown,Unknown,...,None,None,None,None,POINT (-105.25315 40.00249),0,38,City,7.138182e+07,45971.753983
49824,124246,TREE292270,Unknown,Cottonwood,Unknown,None,None,Cottonwood unknown species,Unknown,None,...,None,None,None,None,POINT (-105.23887 39.97840),0,38,City,7.138182e+07,45971.753983
49825,124247,TREE292272,Unknown,Unknown,Unknown,Unknown,None,"Cottonwood, Eastern",Unknown,Unknown,...,None,None,None,None,POINT (-105.23885 39.97837),0,38,City,7.138182e+07,45971.753983
49826,124646,TREE292271,Unknown,Unknown,Unknown,Unknown,None,Crabapple unknown species,Unknown,Unknown,...,None,None,None,None,POINT (-105.23887 39.97838),0,38,City,7.138182e+07,45971.753983


In [32]:
# Plot the tree data
# ---

(
    trees_gdf.hvplot(
        geo=True, color='forestgreen', alpha=.01)
    * boulder_gdf.hvplot(
        geo=True, fill_color=None)
)

:Overlay
   .Points.I   :Points   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

## Investigate the data

In [39]:
# Plot the tree data next to LST
# ---

boulder_plot = boulder_gdf.hvplot(
    geo=True, 
    line_color='black', fill_color=None
)
(
    # LST on the left
    (
        boulder_plot
        * boulder_lst_da.hvplot(
            geo=True,
            cmap="plasma",
            clabel="Land Surface Temperature [K]",
            title='Boulder Trees and Land Surface Temperature',
        )
    )
    + 
    # Trees on the right
    (
        boulder_plot
        * trees_gdf.hvplot(
            geo=True, 
            tiles='EsriImagery', 
            color='forestgreen', alpha=.05,
            hover=False)
    )
)

:Layout
   .Overlay.I  :Overlay
      .Polygons.I :Polygons   [Longitude,Latitude]
      .Image.I    :Image   [x,y]   (value)
   .Overlay.II :Overlay
      .Polygons.I :Polygons   [Longitude,Latitude]
      .WMTS.I     :WMTS   [Longitude,Latitude]
      .Points.I   :Points   [Longitude,Latitude]

# Compare Land Surface Temperature near trees and away from trees

In [37]:
# Find the LST nearest each tree location
# ---

trees_gdf['LST'] = trees_gdf.geometry.apply(
    lambda geom: boulder_lst_da.sel(x=geom.x, y=geom.y, method="nearest").values)
trees_gdf

,OBJECTID_left,FACILITYID,SPECIESCD,FULLNAME,LATINNAME,GENUS,CULTIVAR,COMMONNAME,GENUSCOM,LEAFCYCLE,...,DIVERSITYI,SPECIESIND,OTHERINDEX,geometry,index_right,OBJECTID_right,TYPE,ShapeSTAre,ShapeSTLen,LST
0,1,TREE79142,FRPE,Fraxinus pennsylvanica,Fraxinus pennsylvanica,Fraxinus,None,"Ash, Green",Ash,Deciduous,...,None,None,None,POINT (-105.22100 40.02395),0,38,City,7.138182e+07,45971.753983,295.119995
1,2,TREE79143,FRPE,Fraxinus pennsylvanica,Fraxinus pennsylvanica,Fraxinus,None,"Ash, Green",Ash,Deciduous,...,None,None,None,POINT (-105.22103 40.02378),0,38,City,7.138182e+07,45971.753983,295.119995
2,3,TREE79144,CASP,Catalpa speciosa,Catalpa speciosa,Catalpa,None,"Catalpa, Western",Catalpa,Deciduous,...,None,None,None,POINT (-105.22109 40.02361),0,38,City,7.138182e+07,45971.753983,295.119995
3,4,TREE79145,CASP,Catalpa speciosa,Catalpa speciosa,Catalpa,None,"Catalpa, Western",Catalpa,Deciduous,...,None,None,None,POINT (-105.22116 40.02349),0,38,City,7.138182e+07,45971.753983,295.160004
4,5,TREE79146,FRAM,Fraxinus americana,Fraxinus americana,Fraxinus,None,"Ash, White",Ash,Deciduous,...,None,None,None,POINT (-105.22174 40.02329),0,38,City,7.138182e+07,45971.753983,296.260010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49823,119846,TREE292192,Unknown,Unknown,Unknown,Unknown,None,"Willow, Crack",Unknown,Unknown,...,None,None,None,POINT (-105.25315 40.00249),0,38,City,7.138182e+07,45971.753983,296.440002
49824,124246,TREE292270,Unknown,Cottonwood,Unknown,None,None,Cottonwood unknown species,Unknown,None,...,None,None,None,POINT (-105.23887 39.97840),0,38,City,7.138182e+07,45971.753983,293.000000
49825,124247,TREE292272,Unknown,Unknown,Unknown,Unknown,None,"Cottonwood, Eastern",Unknown,Unknown,...,None,None,None,POINT (-105.23885 39.97837),0,38,City,7.138182e+07,45971.753983,293.000000
49826,124646,TREE292271,Unknown,Unknown,Unknown,Unknown,None,Crabapple unknown species,Unknown,Unknown,...,None,None,None,POINT (-105.23887 39.97838),0,38,City,7.138182e+07,45971.753983,293.000000


In [38]:
# Plot LST distribution for trees vs. total LST distribution
# ---

(
    trees_gdf.LST.hvplot.violin(
        label='Near Trees', violin_fill_color='forestgreen', box_fill_color='forestgreen')
    * boulder_lst_da.to_dataframe(name='LST').LST.hvplot.violin(
        label='Total', violin_fill_color='skyblue', box_fill_color='skyblue')
)

:Overlay
   .Violin.Near_Trees :Violin   (LST)
   .Violin.Total      :Violin   (LST)

## What do you observe?

*Write your response here*